In [1]:
# import useful libraries

import numpy as np
import itertools
from collections import Counter
from utils import load_puzzle_input, test_code
import networkx as nx

In [2]:
# load puzzle input
    
puzzle_input = load_puzzle_input('input.txt')

print(puzzle_input[:3])

['abcccccccaaaaaaaaccccccccccaaaaaaccccccaccaaaaaaaccccccaacccccccccaaaaaaaaaaccccccccccccccccccccccccccccccccaaaaa', 'abcccccccaaaaaaaaacccccccccaaaaaacccccaaacaaaaaaaaaaaccaacccccccccccaaaaaaccccccccccccccccccccccccccccccccccaaaaa', 'abcccccccaaaaaaaaaaccccccccaaaaaacaaacaaaaaaaaaaaaaaaaaaccccccccccccaaaaaaccccccccccccccaaacccccccccccccccccaaaaa']


### Part 1

In [3]:
# code for preparing and testing examples for part 1

example_dict_part_1 = {
    tuple(load_puzzle_input('example_1.txt')): 31
}

In [4]:
def convert_character_to_elevation(character):
    if character == 'S':
        return 1
    elif character == 'E':
        return 26
    return ord(character) - 96

In [5]:
def parse_puzzle_input(puzzle_input):
    return np.array([[convert_character_to_elevation(character) for character in line] for line in puzzle_input])

In [6]:
def identify_start_and_finish(puzzle_input):
    key_points = {}
    for row_idx, row in enumerate(puzzle_input):
        for col_idx, character in enumerate(row): 
            if character in {'S', 'E'}:
                key_points[character] = (row_idx, col_idx)
    return key_points['S'], key_points['E']

In [40]:
def check_valid_step(elevation_a, elevation_b):
    if elevation_b < elevation_a:
        return True
    return elevation_b in {elevation_a, elevation_a + 1}

In [42]:
def generate_link_dict(elevation_array):
    """ Dictionary of possible links from each coordinate """
    
    link_dict = {}
    
    max_row, max_col = elevation_array.shape
    max_row -= 1 
    max_col -= 1
     
    for row_idx, row in enumerate(elevation_array):
        for col_idx, elevation in enumerate(row):
            
            link_dict[(row_idx, col_idx)] = []
            
            # look up
            if row_idx != 0:
                
                up_elevation = elevation_array[row_idx - 1, col_idx]
                if check_valid_step(elevation, up_elevation):
                    link_dict[(row_idx, col_idx)].append((row_idx - 1, col_idx))
                    
            # look down
            if row_idx != max_row:
                
                down_elevation = elevation_array[row_idx + 1, col_idx]
                if check_valid_step(elevation, down_elevation):
                    link_dict[(row_idx, col_idx)].append((row_idx + 1, col_idx))
                    
            # look right 
            if col_idx != max_col:
                
                right_elevation = elevation_array[row_idx, col_idx + 1]
                if check_valid_step(elevation, right_elevation):
                    link_dict[(row_idx, col_idx)].append((row_idx, col_idx + 1))

                    
            # look left 
            if col_idx != 0:
                
                left_elevation = elevation_array[row_idx, col_idx - 1]
                if check_valid_step(elevation, left_elevation):
                    link_dict[(row_idx, col_idx)].append((row_idx, col_idx - 1))
                    
    return link_dict


In [56]:
# why reinvent the wheel. go networkx!

def find_shortest_path(start, end, link_dict):
    
    try: # gotta catch that no path error somehow lol
        Graph = nx.DiGraph()
        
        for node in link_dict:
            Graph.add_node(node)
            
        for node in link_dict:
            for next_node in link_dict[node]:
                Graph.add_edge(node, next_node)
                
        # print(Graph.nodes)
        # print(Graph.edges)
                
        # print(nx.shortest_path(Graph, source = start, target = end, weight = None, method = 'dijkstra'))
        
        return nx.shortest_path_length(Graph, source = start, target = end, weight = None, method = 'dijkstra')
    except:
        return 10**10 # omg I'm so done with this, just a really high number
        

In [45]:
def part_1_solution(puzzle_input):
    
    elevation_array = parse_puzzle_input(puzzle_input)
    start_point, end_point = identify_start_and_finish(puzzle_input)

    # print(start_point)
    # print(end_point)

    link_dict = generate_link_dict(elevation_array)

    return find_shortest_path(start_point, end_point, link_dict)

In [46]:
# test part 1 solution

test_code(part_1_solution, example_dict_part_1)

Test 0 passed: Input <('Sabqponm', 'abcryxxl', 'accszExk', 'acctuvwj', 'abdefghi')> gives output <31>.

Congratulations! Looks like you cracked it! Good job!


In [47]:
part_1_solution(puzzle_input)

380

### Part 2

In [48]:
# code for preparing and testing examples for part 2

example_dict_part_2 = {
    tuple(load_puzzle_input('example_1.txt')): 29
}

In [50]:
def identify_start_points(puzzle_input):
    key_points = []
    for row_idx, row in enumerate(puzzle_input):
        key_points.extend((row_idx, col_idx) for col_idx, character in enumerate(row) if character in {'S', 'a'})
    return key_points

In [51]:
def part_2_solution(puzzle_input):
    
    elevation_array = parse_puzzle_input(puzzle_input)
    _, end_point = identify_start_and_finish(puzzle_input)
    start_points_list = identify_start_points(puzzle_input)

    link_dict = generate_link_dict(elevation_array)

    step_list = [find_shortest_path(start_point, end_point, link_dict) for start_point in start_points_list]

    return min(step_list)

In [52]:
# test part 2 solution

test_code(part_2_solution, example_dict_part_2)

Test 0 passed: Input <('Sabqponm', 'abcryxxl', 'accszExk', 'acctuvwj', 'abdefghi')> gives output <29>.

Congratulations! Looks like you cracked it! Good job!


In [57]:
part_2_solution(puzzle_input)

375